In [57]:
import geopandas as gpd
from shapely.geometry import LineString
from shapely import Point
import rasterio
from rasterio.sample import sample_gen
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np
from osgeo import gdal

In [39]:
file_path = r"C:\Users\richmond\AMH Philippines, Inc\PP23.307 Rockwell and Roll - My Documents\Python\amh_pyqgis\flow_extraction\GIS\Reference Lines SHP\ref_lines_v2.shp"
gdf = gpd.read_file(file_path)
raster_path = r"C:\Users\richmond\AMH Philippines, Inc\PP23.307 Rockwell and Roll - My Documents\Python\amh_pyqgis\flow_extraction\RAS\Terrain\Terrain.clipped_ifsar.tif"


In [47]:
# Reproject Shapefile Layer
target_crs = "EPSG:32651"
gdf_reprojected = gdf.to_crs(target_crs)

In [65]:
def reproject(vlayer, rlayer):
    # Reproject Shapefile Layer
    target_crs = "EPSG:32651"
    gdf = gpd.read_file(vlayer)
    gpd_reprojected = gdf.to_crs(target_crs)

    # Reproject Raster
    with gdal.Open(rlayer) as src_ds:
        # Create an in-memory dataset for the output
        mem_driver = gdal.GetDriverByName("MEM")
        dst_ds = mem_driver.Create("", src_ds.RasterXSize, src_ds.RasterYSize, src_ds.RasterCount, src_ds.GetRasterBand(1).DataType)

        # Reproject the raster into the in-memory dataset
        gdal.Warp(
            dst_ds,  # Output dataset (in memory)
            src_ds,  # Input dataset
            dstSRS=target_crs  # Target CRS
        )
    return gpd_reprojected, dst_ds


In [66]:
def get_lowest_elevation(line):
    line= LineString(line)
    spacing = 1
    num_points = int(line.length / spacing) + 1
    points = [line.interpolate(i * spacing) for i in range(num_points)]
    point_coords = np.array([(point.x, point.y) for point in points])

    with rasterio.open(rlayer, 'r') as src:
        min_elevation = min(list(sample_gen(src, point_coords)))

    return min_elevation.item()



In [69]:
df = reproject(file_path, raster_path)[0]
rlayer = reproject(file_path, raster_path)[1]

In [70]:
df['thalweg'] = df['geometry'].apply(
                                lambda line: get_lowest_elevation(line)
)

TypeError: invalid path or file: <osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000001CA0251E850> >